## Part 2.2

### Import data and modules

In [1]:
import numpy as np
import networkx as nx
import pandas as pd
from scipy.stats import hypergeom
import gseapy
import re
import subprocess

In [2]:
df_seed_genes = pd.read_csv("Seed_Genes.csv", sep = "\t")
df_seed_genes.head()

,seed Gene,Uniprot_AC,protein_name,entrez_gene_id,description
0,IFNG,P01579,Interferon gamma,3458,Produced by lymphocytes activated by specific ...
1,CD28,P10747,T-cell-specific surface glycoprotein CD28,940,"Involved in T-cell activation, the induction o..."
2,HLA-B,P01889,"HLA class I histocompatibility antigen, B alph...",3106,Antigen-presenting major histocompatibility co...
3,KIR3DL1,P43629,Killer cell immunoglobulin-like receptor 3DL1,3811,Receptor on natural killer (NK) cells for HLA ...
4,HLA-C,P10321,"HLA class I histocompatibility antigen, C alph...",3107,Antigen-presenting major histocompatibility co...


In [3]:
df_seed_interactome = pd.read_csv("SeedGeneInteractome.csv", sep = "\t")
df_seed_interactome.head()

,interactor A gene symbol,interactor B gene symbol,interactor A Uniprot AC,interactor B Uniprot AC,database source
0,IFNG,IFNG,P01579,P01579,Biogrid
1,HLA-B,HLA-B,P01889,P01889,Biogrid
2,HLA-B,ADRB2,P01889,P07550,Biogrid
3,HLA-B,HLA-C,P01889,P10321,Biogrid
4,KIR3DL1,KIR3DL1,P43629,P43629,Biogrid


In [4]:
df_union_interactome = pd.read_csv("UnionInteractome.csv", sep = "\t")
df_union_interactome.head()

,interactor A gene symbol,interactor B gene symbol,interactor A Uniprot AC,interactor B Uniprot AC
0,IFNG,RP3-503F13.3,P01579,-
1,IFNG,GOPC,P01579,Q9HD26
2,IFNG,IFNGR2,P01579,P38484
3,IFNG,STAT6,P01579,P42226
4,IFNG,IFNG,P01579,P01579


In [5]:
df_intersection_interactome = pd.read_csv("IntersectionInteractome.csv", sep = "\t")
df_intersection_interactome.head()

,interactor A gene symbol,interactor B gene symbol,interactor A Uniprot AC,interactor B Uniprot AC
0,DHX15,ARRB2,O43143,P32121
1,CAD,HDAC6,P27708,Q9UBN7
2,RPL7,RPS13,P18124,P62277
3,MYC,HSP90AA1,P01106,P07900
4,TP53,GNL3,P04637,Q9BVP2


### U-LCC modules analysis

In [6]:
# Create union interactome graph
G_union = nx.Graph()
edge_list = np.append(np.array(df_union_interactome["interactor A gene symbol"]).reshape(-1,1), 
                      np.array(df_union_interactome["interactor B gene symbol"]).reshape(-1,1), 
                      axis = 1)
G_union.add_edges_from(edge_list)

# Find largest connected component
conn_comps = list(nx.connected_component_subgraphs(G_union))

max_idx = 0
for idx in range(len(conn_comps)):
    if len(conn_comps[idx].nodes) > len(conn_comps[max_idx].nodes):
        max_idx = idx

U_LCC = conn_comps[max_idx]

In [7]:
df_U_LCC = df_union_interactome[["interactor A gene symbol", "interactor B gene symbol"]][df_union_interactome["interactor A gene symbol"].isin(list(U_LCC.nodes)) & df_union_interactome["interactor B gene symbol"].isin(list(U_LCC.nodes))]
df_U_LCC = df_U_LCC.reset_index(drop = True)

In [8]:
# Write input file for mcl-edge
with open('modules/input_ULCC.abc', 'w', newline='') as f:
    for idx in range(len(df_U_LCC)):
        f.write('{} {}\n'.format(df_U_LCC["interactor A gene symbol"][idx],df_U_LCC["interactor B gene symbol"][idx]))
    f.close()
    
# Convert file in native mcl-edge format    
!./mcxload --stream-mirror -abc modules/input_ULCC.abc -o modules/input_ULCC.mci

.....[mclIO] writing <modules/input_ULCC.mci>
.......................................
[mclIO] wrote native interchange 5623x5623 matrix with 216619 entries to stream <modules/input_ULCC.mci>


In [9]:
# Perform a "grid search" for different values of the inflation parameter
# looking for the best efficiency

inflation_list = ["1.2", "1.5", "2", "2.5", "3", "3.5", "4", "4.5", "5"]

# for each inflation value
for i_val in inflation_list:
    # perform the clustering
    cmd = './mcl modules/input_ULCC.mci -I ' + i_val + ' -o modules/output_ULCC.mci'
    cmd = cmd.split(" ")
    subprocess.run(cmd)
    # compute metrics and get output
    cmd = './clm info modules/input_ULCC.mci modules/output_ULCC.mci'
    cmd = cmd.split(" ")
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    popen.wait()
    output = popen.stdout.read()
    output = str(output)
    # obtain efficiency
    eff = output.split(" ")[0]
    eff = re.search("[=].+", eff)[0].replace("=", "")
    # obtain number of clusters
    ncl = output.split(" ")[5]
    ncl = re.search("[=].+", ncl)[0].replace("=", "")
    print("inflation: {};    efficiency: {};    n. clusters: {}".format(i_val, eff, ncl))

inflation: 1.2;    efficiency: 0.00699;    n. clusters: 1
inflation: 1.5;    efficiency: 0.03846;    n. clusters: 58
inflation: 2;    efficiency: 0.05890;    n. clusters: 390
inflation: 2.5;    efficiency: 0.07154;    n. clusters: 1625
inflation: 3;    efficiency: 0.07755;    n. clusters: 2376
inflation: 3.5;    efficiency: 0.08070;    n. clusters: 2726
inflation: 4;    efficiency: 0.08191;    n. clusters: 2955
inflation: 4.5;    efficiency: 0.08251;    n. clusters: 3078
inflation: 5;    efficiency: 0.08263;    n. clusters: 3178


In [10]:
# perform the final clustering, with the chosen inflation value
!./mcl "modules/input_ULCC.abc" -I 3.5 --abc -o "modules/output_ULCC"

.....[mcl] new tab created
[mcl] pid 9473
 ite -------------------  chaos  time hom(avg,lo,hi) m-ie m-ex i-ex fmv
  1  ................... 208.04  2.01 1.15/0.00/7.12 40.34 24.52 24.52  75
  2  ................... 110.14 39.53 0.66/0.20/1.45 5.74 0.14 3.47 100
  3  ...................  13.44  0.94 0.89/0.12/1.38 9.67 0.05 0.16  76
  4  ...................   2.53  0.02 0.95/0.39/1.52 1.48 0.31 0.05   1
  5  ...................   1.35  0.01 0.98/0.56/1.19 1.02 0.66 0.03   0
  6  ...................   0.65  0.01 0.99/0.53/1.19 1.00 0.84 0.03   0
  7  ...................   0.47  0.01 1.00/0.56/1.05 1.00 0.93 0.03   0
  8  ...................   0.36  0.01 1.00/0.59/1.00 1.00 0.97 0.03   0
  9  ...................   0.24  0.01 1.00/0.76/1.00 1.00 0.99 0.03   0
 10  ...................   0.25  0.01 1.00/0.77/1.00 1.00 1.00 0.03   0
 11  ...................   0.16  0.00 1.00/0.91/1.00 1.00 1.00 0.03   0
 12  ...................   0.24  0.01 1.00/0.76/1.00 1.00 1.00 0.03   0
 13  ..............

In [11]:
# read mcl-edge output file
f = open("modules/output_ULCC")
lines = f.read()
lines = lines.split("\n")
len(lines)
lines.remove("")
f.close()

In [12]:
# obtain mcl-edge clusters
clusters = []
for line in lines:
    cl = line.split("\t")
    if len(cl) >= 10:
        clusters.append(cl)

In [13]:
# Hypergeometric test for putative disease modules

# N = number of genes in U-LCC
# K = number of genes in U-LCC which are seed genes
# k = number of genes in current module which are seed genes
# n = number of genes in the current module

N = len(U_LCC.nodes)
K = 0
for node in U_LCC.nodes:
    if node in list(df_seed_genes["seed Gene"]):
        K += 1
        
# list of p-values
pval_list = []

cols = ["algorithm", "moduleID", "n seed genes", "n genes", "seed IDs", "genes IDs", "pvalue"]
df_ULCC_results = pd.DataFrame(columns=cols)

# for each cluster
for idx in range(len(clusters)):
    
    k = 0
    seed_ids = []
    # get seeds in cluster
    for node in clusters[idx]:
        if node in list(df_seed_genes["seed Gene"]):
            k += 1
            seed_ids.append(node)
    n = len(clusters[idx])
    
    # perform hypergeometric test
    # and get p-value
    pval = hypergeom.sf(k-1, N, K, n)
    pval_list.append(pval)
    
    # create dataframe row for cluster
    tmp = pd.DataFrame(["mcl", idx, k, n, ", ".join(seed_ids), ", ".join(clusters[idx]), pval]).T
    tmp.columns = cols
    df_ULCC_results = df_ULCC_results.append(tmp)

In [14]:
df_ULCC_results.head()

,algorithm,moduleID,n seed genes,n genes,seed IDs,genes IDs,pvalue
0,mcl,0,2,234,"HSP90AA1, HSP90AB1","RP11-383C12.1, CDKL1, SC22CB-5E3.5, HSP90AA1, ...",0.925569
0,mcl,1,1,135,BAG3,"RP11-290D2.1, MSTP006, LL22NC03-44A4.1, BAG3, ...",0.913846
0,mcl,2,1,87,ADRB2,"ADRB2, XXbac-BCX101P6.2, PRTN3, RP11-353K22.1,...",0.792584
0,mcl,3,1,87,CCT3,"RP23-46N3.2, RP13-46G5.1, RP11-228B15.5, RP11-...",0.792584
0,mcl,4,1,87,MDM2,"RP1-85F18.1, MDM2, RP1-130E4.1, LA16c-358B7.1,...",0.792584


In [15]:
# get putative disease modules
threshold = 0.05
df_ULCC_disease_modules = df_ULCC_results[df_ULCC_results["pvalue"] < threshold]
df_ULCC_disease_modules

,algorithm,moduleID,n seed genes,n genes,seed IDs,genes IDs,pvalue
0,mcl,51,2,14,"ARHGDIA, ARHGDIB","ARHGDIA, RP11-427L11.1, MIG5, HAAO, MSTP011, R...",0.0247857


### I-LCC modules analysis

In [16]:
# Create intersection interactome graph
G_intersection = nx.Graph()
edge_list = np.append(np.array(df_intersection_interactome["interactor A gene symbol"]).reshape(-1,1), 
                      np.array(df_intersection_interactome["interactor B gene symbol"]).reshape(-1,1), 
                      axis = 1)
G_intersection.add_edges_from(edge_list)

# Find largest connected component
conn_comps = list(nx.connected_component_subgraphs(G_intersection))

max_idx = 0
for idx in range(len(conn_comps)):
    if len(conn_comps[idx].nodes) > len(conn_comps[max_idx].nodes):
        max_idx = idx

I_LCC = conn_comps[max_idx]

In [17]:
df_I_LCC = df_intersection_interactome[["interactor A gene symbol", "interactor B gene symbol"]][df_intersection_interactome["interactor A gene symbol"].isin(list(I_LCC.nodes)) & df_intersection_interactome["interactor B gene symbol"].isin(list(I_LCC.nodes))]
df_I_LCC = df_I_LCC.reset_index(drop = True)

In [18]:
# Write input file for mcl-edge
with open('modules/input_ILCC.abc', 'w', newline='') as f:
    for idx in range(len(df_I_LCC)):
        f.write('{} {}\n'.format(df_I_LCC["interactor A gene symbol"][idx],df_I_LCC["interactor B gene symbol"][idx]))
    f.close()

# Convert input file to mcl-edge native format
!./mcxload --stream-mirror -abc modules/input_ILCC.abc -o modules/input_ILCC.mci

[mclIO] writing <modules/input_ILCC.mci>
........................................
[mclIO] wrote native interchange 1120x1120 matrix with 14941 entries to stream <modules/input_ILCC.mci>


In [19]:
# Perform a "grid search" for different values of the inflation parameter
# looking for the best efficiency

# for each inflation value
for i_val in ["1.2", "1.5", "2", "2.5", "3", "3.5", "4", "4.5", "5"]:
    # perform the clustering
    cmd = './mcl modules/input_ILCC.mci -I ' + i_val + ' -o modules/output_ILCC.mci'
    cmd = cmd.split(" ")
    subprocess.run(cmd)
    # compute metrics and get output
    cmd = './clm info modules/input_ILCC.mci modules/output_ILCC.mci'
    cmd = cmd.split(" ")
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE)
    popen.wait()
    output = popen.stdout.read()
    output = str(output)
    # obtain efficiency
    eff = output.split(" ")[0]
    eff = re.search("[=].+", eff)[0].replace("=", "")
    # obtain number of clusters
    ncl = output.split(" ")[5]
    ncl = re.search("[=].+", ncl)[0].replace("=", "")
    print("inflation: {};    efficiency: {};    n. clusters: {}".format(i_val, eff, ncl))

inflation: 1.2;    efficiency: 0.01517;    n. clusters: 2
inflation: 1.5;    efficiency: 0.12460;    n. clusters: 35
inflation: 2;    efficiency: 0.27770;    n. clusters: 192
inflation: 2.5;    efficiency: 0.29656;    n. clusters: 343
inflation: 3;    efficiency: 0.29189;    n. clusters: 451
inflation: 3.5;    efficiency: 0.27708;    n. clusters: 545
inflation: 4;    efficiency: 0.26974;    n. clusters: 609
inflation: 4.5;    efficiency: 0.26232;    n. clusters: 650
inflation: 5;    efficiency: 0.25650;    n. clusters: 677


In [20]:
# perform the final clustering, with the chosen inflation value
!./mcl "modules/input_ILCC.abc" -I 2.5 --abc -o "modules/output_ILCC"

[mcl] new tab created
[mcl] pid 9511
 ite --------------------  chaos  time hom(avg,lo,hi) m-ie m-ex i-ex fmv
  1  ....................  55.27  0.05 1.02/0.01/3.24 13.14 13.10 13.10  70
  2  ....................  83.10  0.36 0.66/0.19/1.36 5.57 0.98 12.81  99
  3  ....................  16.99  0.22 0.70/0.24/1.48 5.77 0.13 1.72  99
  4  ....................   4.18  0.01 0.85/0.36/1.93 4.04 0.24 0.41  82
  5  ....................   1.93  0.00 0.93/0.36/1.19 1.36 0.42 0.17   6
  6  ....................   0.64  0.00 0.97/0.53/1.21 1.06 0.63 0.11   0
  7  ....................   0.37  0.00 0.99/0.58/1.26 1.04 0.75 0.08   0
  8  ....................   0.24  0.00 1.00/0.52/1.00 1.00 0.93 0.07   0
  9  ....................   0.25  0.00 1.00/0.76/1.00 1.00 0.97 0.07   0
 10  ....................   0.23  0.00 1.00/0.77/1.00 1.00 0.99 0.07   0
 11  ....................   0.25  0.00 1.00/0.76/1.00 1.00 0.99 0.07   0
 12  ....................   0.09  0.00 1.00/0.91/1.00 1.00 1.00 0.07   0
 13  .....

In [21]:
# read mcl-edge output file
f = open("modules/output_ILCC")
lines = f.read()
lines = lines.split("\n")
len(lines)
lines.remove("")
f.close()

In [22]:
# obtain mcl-edge clusters
clusters = []
for line in lines:
    cl = line.split("\t")
    if len(cl) >= 10:
        clusters.append(cl)

In [23]:
# Hypergeometric test for putative disease modules

# N = number of genes in I-LCC
# K = number of genes in I-LCC which are seed genes
# k = number of genes in current module which are seed genes
# n = number of genes in the current module

N = len(I_LCC.nodes)
K = 0
for node in I_LCC.nodes:
    if node in list(df_seed_genes["seed Gene"]):
        K += 1

# p-values list
pval_list = []

cols = ["algorithm", "moduleID", "n seed genes", "n genes", "seed IDs", "genes IDs", "pvalue"]
df_ILCC_results = pd.DataFrame(columns=cols)

# for each cluster
for idx in range(len(clusters)):
    
    k = 0
    # get seeds in cluster
    seed_ids = []
    for node in clusters[idx]:
        if node in list(df_seed_genes["seed Gene"]):
            k += 1
            seed_ids.append(node)
    n = len(clusters[idx])
    
    # perform hypergeometric test
    # and get p-value
    pval = hypergeom.sf(k-1, N, K, n)
    pval_list.append(pval)
    
    # create dataframe row for cluster
    tmp = pd.DataFrame(["mcl", idx, k, n, ", ".join(seed_ids), ", ".join(clusters[idx]), pval]).T
    tmp.columns = cols
    df_ILCC_results = df_ILCC_results.append(tmp)

In [24]:
df_ILCC_results.head()

,algorithm,moduleID,n seed genes,n genes,seed IDs,genes IDs,pvalue
0,mcl,0,0,48,,"TP53, CHEK1, ATM, SP1, PARP1, TP63, CHD3, VDR,...",1
0,mcl,1,0,40,,"GRB2, ARHGAP17, BLNK, SYK, BCR, CBL, IGF1R, HC...",1
0,mcl,2,8,31,"PSMA6, PSMC3, PSMB4, PSMD3, PSMB10, PSMD8, PSM...","PSMC1, ADRM1, USP14, PSMD11, PSMC2, PSMA1, PSM...",0.000426505
0,mcl,3,0,28,,"JUND, EP300, JUN, SMAD4, NFATC2, FOXO3, FOXO1,...",1
0,mcl,4,1,25,COPS6,"COPS7A, DDB1, CUL5, COPS3, COPS5, CUL4B, DDB2,...",0.80904


In [25]:
# get putative disease modules
threshold = 0.05
df_ILCC_disease_modules = df_ILCC_results[df_ILCC_results["pvalue"] < threshold]
df_ILCC_disease_modules

,algorithm,moduleID,n seed genes,n genes,seed IDs,genes IDs,pvalue
0,mcl,2,8,31,"PSMA6, PSMC3, PSMB4, PSMD3, PSMB10, PSMD8, PSM...","PSMC1, ADRM1, USP14, PSMD11, PSMC2, PSMA1, PSM...",0.000426505


In [26]:
df_ULCC_disease_modules.to_csv("modules/df_ULCC_disease_modules.csv", index = False)

In [27]:
df_ILCC_disease_modules.to_csv("modules/df_ILCC_disease_modules.csv", index = False)

In [43]:
# create summary table including ratio between n. of seed genes and total genes
df_ULCC_table = df_ULCC_disease_modules[["n seed genes", "n genes", "pvalue"]]
df_ULCC_table["ratio"] = df_ULCC_table["n seed genes"]/df_ULCC_table["n genes"]
df_ILCC_table = df_ILCC_disease_modules[["n seed genes", "n genes", "pvalue"]]
df_ILCC_table["ratio"] = df_ILCC_table["n seed genes"]/df_ILCC_table["n genes"]

/home/jagg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/jagg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [44]:
df_ULCC_table

,n seed genes,n genes,pvalue,ratio
0,2,14,0.0247857,0.142857


In [45]:
df_ILCC_table

,n seed genes,n genes,pvalue,ratio
0,8,31,0.000426505,0.258065


In [46]:
df_ULCC_table.to_csv("modules/df_ULCC_table.csv", index = False)
df_ILCC_table.to_csv("modules/df_ILCC_table.csv", index = False)